In [3]:
import pandas as pd
# 'data_csv' 폴더를 경로에 추가합니다.
df1 = pd.read_csv('../data_csv/11_11/강원도업종.csv', encoding = 'CP949')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   업종      356 non-null    object
 1   업소명     356 non-null    object
 2   주소      356 non-null    object
 3   연락처     351 non-null    object
dtypes: object(4)
memory usage: 11.3+ KB


In [6]:
df2 = df1[df1['주소'].str.contains('춘천시')]
df2.to_csv('../data_csv/11_11/춘천시업종.csv')

In [13]:
df2.columns.to_list()

['업종', '업소명', '주소', '연락처']

In [7]:
import pandas as pd
import requests
import time

def get_lat_lon_from_address(address, api_key):
    """
    카카오맵 API를 사용해 주소를 위도와 경도로 변환하는 함수
    
    Args:
        address (str): 변환할 주소 (도로명 또는 지번)
        api_key (str): 카카오 디벨로퍼스에서 발급받은 REST API 키

    Returns:
        tuple: (위도, 경도) 튜플. 실패 시 (None, None) 반환.
    """
    KAKAO_API_URL = "https://dapi.kakao.com/v2/local/search/address.json"
    
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    
    params = {
        "query": address
    }
    
    try:
        response = requests.get(KAKAO_API_URL, headers=headers, params=params, timeout=5)
        response.raise_for_status() # 오류 발생 시 예외 발생
        
        data = response.json()
        
        # 'documents' 리스트가 비어있지 않고, 첫 번째 결과가 있는지 확인
        if data.get('documents'):
            # 카카오 API는 위도를 'y', 경도를 'x'로 반환합니다.
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            return (latitude, longitude)
        else:
            print(f"주소 변환 실패 (결과 없음): {address}")
            return (None, None)
            
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {address} ({e})")
        return (None, None)
    except Exception as e:
        print(f"기타 오류: {address} ({e})")
        return (None, None)



In [14]:
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가
import os
# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("kakao_reset_key") 
# 1. 파일 읽기
df = pd.read_csv("../data_csv/11_11/춘천시업종.csv")



# 3. 위도, 경도 변환 적용
# '소재지(도로명)' 컬럼을 기준으로 변환합니다. 
# 만약 도로명 주소가 비어있는 경우 '소재지(지번)'을 사용하도록 할 수도 있습니다.
# 여기서는 '소재지(도로명)'을 우선 사용합니다.
# 참고: 데이터가 많을 경우 시간이 매우 오래 걸릴 수 있습니다.

# 결과를 저장할 리스트 초기화
latitudes = []
longitudes = []

print("주소 변환을 시작합니다. 데이터 양에 따라 시간이 오래 걸릴 수 있습니다...")

for index, row in df.iterrows():
    address_to_search = row['주소']
    
    # 만약 '소재지(도로명)'이 비어있다면 '소재지(지번)' 사용 (선택적)
    if pd.isna(address_to_search) or address_to_search.strip() == "":
        address_to_search = row['소재지(지번)']
        
    if pd.isna(address_to_search):
        lat, lon = None, None
    else:
        # API 무료 할당량(쿼터) 초과를 방지하기 위해 약간의 지연 시간(sleep)을 줍니다.
        # time.sleep(0.01) # 1초에 100회 미만으로 조절
        
        lat, lon = get_lat_lon_from_address(address_to_search, api_key)

    latitudes.append(lat)
    longitudes.append(lon)
    
    if (index + 1) % 100 == 0:
        print(f"{index + 1} / {len(df)} 건 처리 중...")

# 4. DataFrame에 새로운 컬럼 추가
df['위도'] = latitudes
df['경도'] = longitudes

print("주소 변환 완료!")

# 5. 결과 파일로 저장
output_filename = "../data_csv/11_11/춘천시업종_좌표포함.csv"
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"'{output_filename}' 파일로 저장이 완료되었습니다.")
print(df[['업종', '업소명', '주소', '연락처', '위도', '경도']].head())

주소 변환을 시작합니다. 데이터 양에 따라 시간이 오래 걸릴 수 있습니다...
주소 변환 실패 (결과 없음): 강원도 춘천시 명동길29번지 4-5
주소 변환 실패 (결과 없음): 강원도 춘천시 공지로 392-6(약사동)
주소 변환 완료!
'../data_csv/11_11/춘천시업종_좌표포함.csv' 파일로 저장이 완료되었습니다.
   업종      업소명                     주소           연락처                위도  \
0  한식       청재     강원도 춘천시 서면 박사로 927  033-255-2867  37.8984653703934   
1  한식    두리감자탕      강원도 춘천시 후평동 748-4  033-244-2847  37.8884848958585   
2  한식  약사촌 닭갈비       강원도 춘천시 공지로384-9  033-256-9255  37.8719511821893   
3  한식      산마루  강원도 춘천시 백령로 171-1(1층)  033-241-0769  37.8762586263488   
4  한식       청재     강원도 춘천시 서면 박사로 927  033-255-2867  37.8984653703934   

                 경도  
0  127.693760728915  
1   127.75147838385  
2  127.724553429217  
3  127.745220491838  
4  127.693760728915  
